# Speeh Data Analysis

In [38]:
import os
import librosa
import random
import torch
import evaluate
import torchaudio
import numpy as np
import pandas as pd
from datasets import Dataset
from dataclasses import dataclass
from typing import Any, Dict, List, Union
from transformers import Seq2SeqTrainer
from transformers import WhisperTokenizer
from model.feature_extraction_whisper import WhisperFeatureExtractor

In [39]:
# common_voice = Data
model_type = "openai/whisper-small"
mp3_path = "./dataset/cv-corpus-11.0-2022-09-21/bn/clips"
train_tsv = "./dataset/cv-corpus-11.0-2022-09-21/bn/train.tsv"
val_tsv = "./dataset/cv-corpus-11.0-2022-09-21/bn/dev.tsv"

In [40]:
def read_data(data_path, data_chunk = 0):
    if data_chunk==0:
        data = pd.read_csv(data_path, sep='\t')
    else:
        data = pd.read_csv(data_path, sep='\t')[:data_chunk]
    data.columns = ["client_id", "path", "sentence", "up_votes", "down_votes", "age", "gender", "accents", "locale", "segment"]
    data = data.drop(["client_id","up_votes", "down_votes", "age", "gender", "accents", "locale", "segment"], axis=1)
    return data

In [41]:
train_data = read_data(train_tsv, data_chunk=0)
val_data = read_data(val_tsv, data_chunk=0)

In [42]:
train_data.head()

,path,sentence
0,common_voice_bn_31639641.mp3,এই ভোগ সম্পূর্ণভাবে নির্ভর করে তার নিজের উপরে।
1,common_voice_bn_31639700.mp3,নিউ দিল্লী হোটেলস।
2,common_voice_bn_31639731.mp3,এছাড়া মুস-ছেন-সাংস-র্গ্যাস-রিন-ছেন নামক অষ্টম...
3,common_voice_bn_31639763.mp3,পাশাপাশি বাংলাদেশ উন্মুক্ত বিশ্ববিদ্যালয়ে তিন...
4,common_voice_bn_31639767.mp3,শ্রমিকদের উন্নয়নে অসাধারণ ভূমিকার জন্য তাকে হ...


In [43]:
val_data.head()

,path,sentence
0,common_voice_bn_31684551.mp3,"তিনি ভিডিও গেম ""হাফ-লাইফ: বিপক্ষ বলের"" ক্ষেত্র..."
1,common_voice_bn_31684629.mp3,তবে কর্তৃত্ব প্রদর্শনের জন্য এটি জার্মান সেনাদ...
2,common_voice_bn_31581194.mp3,এই লোকসভা কেন্দ্রের সদর দফতর সুলতানপুর শহরে অব...
3,common_voice_bn_31581200.mp3,একটি উপদলের নেতৃত্বে ছিলেন মোহাম্মদ নূরুল হক।
4,common_voice_bn_31549899.mp3,সুস্থ হওয়ার পর তিনি পার্শ্ববর্তী মির্জাপুর গ্...


In [44]:
def add_file_path(path):
    audio_path = os.path.join(mp3_path, path)
    return audio_path

In [45]:
train_data['path'] = train_data['path'].map(lambda x: add_file_path(x))
val_data['path'] = val_data['path'].map(lambda x: add_file_path(x))

In [46]:
train_data

,path,sentence
0,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,এই ভোগ সম্পূর্ণভাবে নির্ভর করে তার নিজের উপরে।
1,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,নিউ দিল্লী হোটেলস।
2,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,এছাড়া মুস-ছেন-সাংস-র্গ্যাস-রিন-ছেন নামক অষ্টম...
3,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,পাশাপাশি বাংলাদেশ উন্মুক্ত বিশ্ববিদ্যালয়ে তিন...
4,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,শ্রমিকদের উন্নয়নে অসাধারণ ভূমিকার জন্য তাকে হ...
...,...,...
16772,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,আবার কেউ বা হাদিস জিজ্ঞাসা করছিল।
16773,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,যা বর্তমানে নতুন মন রাজ্যে দল নামে পরিচিত।
16774,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,দাবার প্রতি তার অন্যরকম অনুরাগ ছিল।
16775,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,"এই জেলার উত্তরে ভারতের মেঘালয় রাজ্য, দক্ষিণে ..."


In [47]:
val_data

,path,sentence
0,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,"তিনি ভিডিও গেম ""হাফ-লাইফ: বিপক্ষ বলের"" ক্ষেত্র..."
1,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,তবে কর্তৃত্ব প্রদর্শনের জন্য এটি জার্মান সেনাদ...
2,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,এই লোকসভা কেন্দ্রের সদর দফতর সুলতানপুর শহরে অব...
3,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,একটি উপদলের নেতৃত্বে ছিলেন মোহাম্মদ নূরুল হক।
4,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,সুস্থ হওয়ার পর তিনি পার্শ্ববর্তী মির্জাপুর গ্...
...,...,...
8348,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,"অন্যদিকে, জিম্বাবুয়ে দল নিউজিল্যান্ডের বিপক্ষ..."
8349,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,দশম শতাব্দীতে বাশকিরিয়ায় ইসলাম ধর্ম বিস্তার ...
8350,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,তাই তিনি কলকাতাতে চলে আসেন এবং সেখানেই বসবাস শ...
8351,./dataset/cv-corpus-11.0-2022-09-21/bn/clips/c...,"ফেরত বিবেচনার অর্থ হল, সাধারণ আইনে, দান করার জ..."


# Generate VOCAB file

In [48]:
def extract_all_chars(batch):
    print(batch)
    all_text = " ".join(batch["sentence"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

In [49]:
train_data_d = Dataset.from_pandas(train_data)
val_data_d = Dataset.from_pandas(val_data)

vocab_train = train_data_d.map(extract_all_chars)
vocab_dev = val_data_d.map(extract_all_chars)

Map:  30%|██████▊                | 5002/16777 [00:00<00:00, 25286.86 examples/s]

{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31639641.mp3', 'sentence': 'এই ভোগ সম্পূর্ণভাবে নির্ভর করে তার নিজের উপরে।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31639700.mp3', 'sentence': 'নিউ দিল্লী হোটেলস।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31639731.mp3', 'sentence': 'এছাড়া মুস-ছেন-সাংস-র্গ্যাস-রিন-ছেন নামক অষ্টম ঙ্গোর-ছেনও তাকে হেবজ্র সম্বন্ধে শিক্ষাদান করেন।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31639763.mp3', 'sentence': 'পাশাপাশি বাংলাদেশ উন্মুক্ত বিশ্ববিদ্যালয়ে তিনি রাজনীতি ও আইন বিষয়ে পাঠদান করেছেন।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31639767.mp3', 'sentence': 'শ্রমিকদের উন্নয়নে অসাধারণ ভূমিকার জন্য তাকে হিউম্যান রাইটস ওয়াচের অ্যালিসন দেস ফোর্স অ্যাওয়ার্ড প্রদান করা হয়েছিল।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31630204.mp3', 'sentence': 'তবে আবার একই জায়গায় মসজিদটি পু

Map:  68%|██████████████▉       | 11384/16777 [00:00<00:00, 25521.66 examples/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30994377.mp3', 'sentence': 'উদাহরণস্বরুপ, তিনি ব্ল্যাক প্যাচ টোব্যাকো যুদ্ধের ন্যায়বিচারের জন্য লড়াই করার প্ল্যাটফর্ম হিসাবে পত্রিকাটি ব্যবহার করেছিলেন।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30994379.mp3', 'sentence': 'তিনি বর্তমানে র ট্যাগ টিম চ্যাম্পিয়নস।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30994380.mp3', 'sentence': 'তিনি ক্রমশ আল জোহরা আল ইব্রাহিমের ওপর নির্ভরশীল হয়ে পড়েন।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30994418.mp3', 'sentence': 'পতনশীল বা আবাদযোগ্য জমি হিসাবে একটি মাঠ থাকতে পারে।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30994421.mp3', 'sentence': 'এর পাশাপাশি, তিনি আরব জগতে হিন্দু সংখ্যা পদ্ধতির প্রচলন ঘটান।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30994425.mp3', 'sentence': 'পরের দিন চুক্তি বাতিল করা হয়েছিল।'}
{'path': './datase

Map:   0%|                                      | 0/8353 [00:00<?, ? examples/s]

{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31684551.mp3', 'sentence': 'তিনি ভিডিও গেম "হাফ-লাইফ: বিপক্ষ বলের" ক্ষেত্রেও উল্লেখ করেছিলেন।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31684629.mp3', 'sentence': 'তবে কর্তৃত্ব প্রদর্শনের জন্য এটি জার্মান সেনাদের উপর অনেকাংশে নির্ভর ছিল।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31581194.mp3', 'sentence': 'এই লোকসভা কেন্দ্রের সদর দফতর সুলতানপুর শহরে অবস্থিত।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31581200.mp3', 'sentence': 'একটি উপদলের নেতৃত্বে ছিলেন মোহাম্মদ নূরুল হক।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31549899.mp3', 'sentence': 'সুস্থ হওয়ার পর তিনি পার্শ্ববর্তী মির্জাপুর গ্রামে হোমিওপ্যাথি চিকিৎসা আরম্ভ করেন।'}
{'path': './dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_31549902.mp3', 'sentence': 'তিনি একটি শোয়ের জন্য রক স্টার হয়ে ভারতে আসেন।'}
{'path': './dataset/cv-c

Map:  33%|███████▊                | 2731/8353 [00:00<00:00, 27190.36 examples/s]IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
len(vocab_train["vocab"])

def create_vocab(vocab_train):
    v_character_list = []
    for v_train in vocab_train["vocab"]:
    #     print(v_train)
        for v_chr in v_train[0]:
            if v_chr not in v_character_list:
                v_character_list.append(v_chr)
    return v_character_list

In [51]:
train_vocab = create_vocab(vocab_train)
val_vocab = create_vocab(vocab_dev)

In [52]:
print(train_vocab)

['র', ' ', 'া', 'স', 'ি', 'গ', 'ন', 'ে', 'ম', 'ই', 'ণ', 'ক', 'এ', 'ব', '।', '্', 'প', 'জ', 'উ', 'ত', 'ো', 'ূ', 'ভ', 'ী', 'ট', 'দ', 'হ', 'ল', '়', 'ঙ', 'শ', 'ষ', 'ও', 'ু', 'ছ', 'ড', 'য', '-', 'ধ', 'অ', 'ং', 'আ', 'ঠ', 'চ', 'ফ', 'খ', 'থ', 'ঘ', 'ড়', 'ৎ', ',', 'ঁ', 'ৃ', 'ঝ', 'ৌ', 'ঋ', 'ৈ', 'য়', 'ঞ', '"', '’', 'ঃ', '!', 'ঈ', ';', 'ঢ', '?', 'ঐ', 'ঊ', "'", '‘', 'ঔ', ':', '.', '/', 'a', 'l', 'P', 'e', 'g', 'y', 'G', 'o', 'ৰ', '—', '‚']


In [53]:
print(val_vocab)

[':', 'র', '"', ' ', 'া', 'ি', 'গ', 'ন', 'ষ', 'খ', 'ও', 'ে', 'ম', 'হ', 'ল', 'ই', 'ছ', 'ক', 'ড', 'ব', '।', '্', 'প', '-', 'ফ', 'উ', 'ত', 'ভ', 'শ', 'দ', 'স', 'ট', 'ৃ', 'এ', 'জ', 'য', 'অ', 'ং', 'ু', 'থ', 'ো', 'ূ', '়', 'আ', 'ী', 'চ', 'ৎ', ',', 'ধ', 'ণ', 'ঘ', 'ঞ', 'ঁ', 'ঠ', 'ৌ', 'ঝ', 'ঙ', 'ঢ', 'ৈ', 'ঃ', 'ঐ', 'ঈ', 'য়', 'ড়', '’', '‘', '?', "'", '/', 'ঊ', 'ঋ', 'ঔ', '!', ';', '॥', 'ৰ', 'ঢ়']


In [54]:
vocab_list = list(set(train_vocab) | set(val_vocab))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)

{'র': 0, '"': 1, 'ঙ': 2, 'g': 3, 'দ': 4, 'গ': 5, '’': 6, 'ন': 7, 'a': 8, 'ও': 9, '‘': 10, 'P': 11, 'ী': 12, 'ৰ': 13, '/': 14, 'ঝ': 15, 'ঘ': 16, '‚': 17, 'এ': 18, 'চ': 19, '।': 20, 'থ': 21, 'জ': 22, 'অ': 23, 'e': 24, 'উ': 25, '!': 26, 'ঁ': 27, 'ঢ়': 28, 'আ': 29, ';': 30, 'ঋ': 31, 'ঊ': 32, 'ষ': 33, 'ু': 34, 'ে': 35, 'ট': 36, 'হ': 37, 'ছ': 38, 'ৈ': 39, '.': 40, 'ঢ': 41, 'ক': 42, '্': 43, 'ঔ': 44, 'ড়': 45, "'": 46, 'ফ': 47, 'ত': 48, 'ো': 49, 'ূ': 50, 'ৎ': 51, 'ভ': 52, 'l': 53, 'শ': 54, 'া': 55, 'ি': 56, 'ঈ': 57, 'ল': 58, 'ই': 59, 'ণ': 60, 'ৃ': 61, '?': 62, 'ড': 63, 'ব': 64, ',': 65, 'প': 66, '-': 67, 'ধ': 68, 'ঠ': 69, 'ঃ': 70, 'ঞ': 71, ':': 72, '়': 73, 'ঐ': 74, ' ': 75, 'স': 76, 'G': 77, 'খ': 78, 'ম': 79, '॥': 80, 'য়': 81, '—': 82, 'য': 83, 'ৌ': 84, 'ং': 85, 'y': 86, 'o': 87}


In [65]:
for i in train_data_d["path"]:
    try:
        speech_array, sampling_rate = torchaudio.load(i, format="mp3")
    except:
        continue
# path_here = "./dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30999412.mp3"


formats: can't open input file `./dataset/cv-corpus-11.0-2022-09-21/bn/clips/common_voice_bn_30999412.mp3': 


AttributeError: 'Dataset' object has no attribute 'head'